# 2.2 - Web Scraping (bs4)

$$$$

![scraping](images/scraping.png)

$$$$

Web scraping o raspado web, es una técnica utilizada mediante programas de software para extraer información de sitios web. Usualmente, estos programas simulan la navegación de un humano en la web ya sea utilizando el protocolo HTTP manualmente, o incrustando un navegador en una aplicación.

El web scraping está muy relacionado con la indexación de la web, la cual indexa la información de la web utilizando un robot y es una técnica universal adoptada por la mayoría de los motores de búsqueda. Sin embargo, el web scraping se enfoca más en la transformación de datos sin estructura en la web, como el formato HTML, en datos estructurados que pueden ser almacenados y analizados en una base de datos central, en una hoja de cálculo o en alguna otra fuente de almacenamiento. Alguno de los usos del web scraping son la comparación de precios en tiendas, la monitorización de datos relacionados con el clima de cierta región, la detección de cambios en sitios webs y la integración de datos en sitios webs. 

En los últimos años el web scraping se ha convertido en una técnica muy utilizada dentro del sector del posicionamiento web gracias a su capacidad de generar grandes cantidades de datos para crear contenidos de calidad.

Podríamos pensar que el web scraping es nuestro último recurso a falta de una API o un feed RSS. A falta de una fuente de datos, siempre podemos extraer aquello que sale por pantalla.

### Extracción desde el HTML

Para scrapear una página web, en primer lugar debemos conocer las estructura que tiene el HTML. Veamos la estructura básica.

El HTML consiste en contenido `<etiquetado>`, es como si fueran cajas de contenido, organizado de manera jerárquica:

```
<html>
    <head>
        <title>Titulo de la pagina</title>
    </head>
    <body>
        <h1>Cabecera</h1>
        <p>Parrafo</p>
    </body>
</html>
```

$$$$

Las etiquetas el HTML se pueden clasificar en varios grupos, dependiendo del tipo de contenido que posea. Estos son algunos ejemplos:

+ cabecera: `<h1>`, `<h2>`, `<h3>`, `<hgroup>`...
+ texto: `<b>`, `<p>`...
+ embebido: `<audio>`, `<img>`, `<video>`...
+ tabular: `<table>`, `<tr>`, `<td>`, `<tbody>`...
+ secciones: `<header>`, `<section>`, `<article>`...
+ metadata: `<meta>`, `<title>`, `<script>`...

$$$$

Las etiquetas pueden tener atributos. Por ejemplo:
 
`<div class="text-monospace" id="name_132", href="www.example.com"> Contenido de la pagina </div>` 

Esta etiqueta `div` tiene los siguientes atributos:

+ class: atributo con valor "text-monospace". La clase no es única en la página.
+ id: atributo con valor "name_132". El id de una etiqueta la identifica de manera unívoca.
+ href: atributo con valor "www.example.com". El href suele contener el link a otra parte de la página.

Siguiendo con la analogía de las cajas, si una etiqueta de HTML es una caja, los atributos serían las pegatinas pegadas en la tapa de la caja.

Conociendo cual es el contenido que queremos extraer, debemos encontrar las etiquetas dentro del HTML de la página web.

Usaremos la herramienta **[BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)**.

In [ ]:
%pip install beautifulsoup4

In [ ]:
import requests as req

from bs4 import BeautifulSoup as bs    # ambos alias son cosa mia

### Ejemplos Wikipedia

**[Países europeos según esperanza de vida](https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy)**

In [ ]:
url='https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy'

In [ ]:
# usamos requests para extraer el HTML

html=req.get(url).content   # o .text

html[:1000]

In [ ]:
len(html)

In [ ]:
# parsear


soup=bs(html, 'html.parser')

type(soup)

In [ ]:
len(soup.find_all('table'))

In [ ]:
tabla=soup.find_all('table')[3]

type(tabla)

In [ ]:
tabla.find_all('tr')[0].text.split('\n')

In [ ]:
filas=tabla.find_all('tr')

filas=[f.text.strip().split('\n') for f in filas]

filas[:5]

In [ ]:
final = []

for f in filas:
    
    tmp=[]
    
    for palabra in f:
        
        if palabra!='':
            tmp.append(palabra)
            
    final.append(tmp)
    
final[:5]

In [ ]:
final = [[palabra for palabra in f if palabra!=''] for f in filas]

final[:5]

In [ ]:
import pandas as pd


col_names = filas[0]

data = filas[1:]


df=pd.DataFrame(data, columns=col_names)

df.head()

In [ ]:
# esto es solo para pandas

df_c = pd.read_clipboard()    # ctrl+V

df_c.head()

In [ ]:
pd.read_clipboard()

### Ejemplo geolocalización por IP

https://tools.keycdn.com/geo

**¿Dónde estoy?**

In [ ]:
url='https://tools.keycdn.com/geo?host=2.136.116.82'

In [ ]:
html = req.get(url).text

sopa = bs(html, 'html.parser')

In [ ]:
sopa.find('div', id='geoResult')

In [ ]:
sopa.find('div', class_='mt-4')

In [ ]:
sopa.find('div', {'id': 'geoResult', 'class': 'mt-4'})

In [ ]:
tabla=sopa.find('div', {'id': 'geoResult', 'class': 'mt-4'})

In [ ]:
tabla.find_all('dd')

In [ ]:
tabla.find_all('dd')[0].text

In [ ]:
[e.text for e in tabla.find_all('dd')]

In [ ]:
conexion=[e.text for e in tabla.find_all('dd')]

In [ ]:
# creo una funcion para todo el proceso, segun ip


def encontrar(ip):
    
    url = f'https://tools.keycdn.com/geo?host={ip}'
    
    html = req.get(url).text

    sopa = bs(html, 'html.parser')
    
    tabla=sopa.find('div', {'id': 'geoResult', 'class': 'mt-4'})
    
    conexion=[e.text for e in tabla.find_all('dd')]
    
    return conexion

In [ ]:
encontrar('168.123.4.5')

In [ ]:
encontrar('46.222.34.200')

### Ejemplo LinkedIn

In [ ]:
url='https://www.linkedin.com/jobs/search/?currentJobId=3443970867&keywords=analista%20de%20datos'

In [ ]:
html = req.get(url).text

sopa = bs(html, 'html.parser')

In [ ]:
tarjetas = sopa.find('ul', class_='jobs-search__results-list')

type(tarjetas)

In [ ]:
len(tarjetas.find_all('li'))

In [ ]:
curro=tarjetas.find_all('li')[0]

titulo=curro.find('h3').text.strip()

empresa = curro.find('h4').text.strip()

link_curro = curro.find('a').attrs['href']

link_comp = curro.find('h4').find('a').attrs['href']

pais = curro.find('span', class_="job-search-card__location").text.strip()

fecha = curro.find('time').attrs['datetime']

In [ ]:
{'titulo': titulo,

'empresa': empresa,

'link_curro': link_curro,

'link_comp': link_comp, 

'pais': pais, 

'fecha': fecha}

In [ ]:

def linkedin(num_pages, keywords, loc, n_secs):

    URL = 'https://www.linkedin.com/jobs/search/'

    data = []

    for i in range(num_pages):

        scrape_url = ''.join([URL,  # url principal
                              f'?keywords={keywords}',
                              '&geoId=105646813'
                              f'&location={loc}',
                              f'&f_TPR={n_secs}',
                              f'&start={i*25}'
                             ])

        print(scrape_url)

        html = req.get(url).text

        sopa = bs(html, 'html.parser')

        tarjetas = sopa.find('ul', class_='jobs-search__results-list')


        for curro in tarjetas:  # bucle sobre los li, las tarjetas de cada curro
            
            try:
                titulo=curro.find('h3').text.strip()

                empresa = curro.find('h4').text.strip()

                link_curro = curro.find('a').attrs['href']

                link_comp = curro.find('h4').find('a').attrs['href']

                pais = curro.find('span', class_="job-search-card__location").text.strip()

                fecha = curro.find('time').attrs['datetime']


                data.append({'titulo': titulo,
                             'empresa': empresa,
                             'link_curro': link_curro,
                             'link_comp': link_comp, 
                             'pais': pais, 
                             'fecha': fecha})
                
            except:
                continue

    return pd.DataFrame(data)

In [ ]:
df=linkedin(5, 'analista%20de%20datos', 'spain', 30000)

In [ ]:
df.head()

In [ ]:
from IPython.display import HTML

HTML(df.to_html(render_links=True))